In [85]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [86]:
df = pd.read_csv('county_data.csv')
df.dropna(subset=['county_fips'], inplace=True)
df.head()

year state_po county_name  county_fips  totalvotes  dvotes  rvotes  \
0  2000       AL     AUTAUGA       1001.0       17208    4942   11993   
1  2000       AL     BALDWIN       1003.0       56480   13997   40872   
2  2000       AL     BARBOUR       1005.0       10395    5188    5096   
3  2000       AL        BIBB       1007.0        7101    2710    4273   
4  2000       AL      BLOUNT       1009.0       17973    4977   12667   

   statevotes  vote_percentage  r_margin_perc  
0     1672551         0.010288       0.004216  
1     1672551         0.033769       0.016068  
2     1672551         0.006215      -0.000055  
3     1672551         0.004246       0.000935  
4     1672551         0.010746       0.004598

In [87]:
with open('maps/county_data.json', 'r') as file:
    topo = json.load(file)

In [88]:
fips_list = df["county_fips"].unique().tolist()
state_county_codes = []
for idx, s in enumerate(fips_list):
    s = str(int(s)).zfill(5)
    state_code = s[0:2]
    county_code = s[2:]
    state_county_codes.append((state_code, county_code))
    
print(state_county_codes)

[('01', '001'), ('01', '003'), ('01', '005'), ('01', '007'), ('01', '009'), ('01', '011'), ('01', '013'), ('01', '015'), ('01', '017'), ('01', '019'), ('01', '021'), ('01', '023'), ('01', '025'), ('01', '027'), ('01', '029'), ('01', '031'), ('01', '033'), ('01', '035'), ('01', '037'), ('01', '039'), ('01', '041'), ('01', '043'), ('01', '045'), ('01', '047'), ('01', '049'), ('01', '051'), ('01', '053'), ('01', '055'), ('01', '057'), ('01', '059'), ('01', '061'), ('01', '063'), ('01', '065'), ('01', '067'), ('01', '069'), ('01', '071'), ('01', '073'), ('01', '075'), ('01', '077'), ('01', '079'), ('01', '081'), ('01', '083'), ('01', '085'), ('01', '087'), ('01', '089'), ('01', '091'), ('01', '093'), ('01', '095'), ('01', '097'), ('01', '099'), ('01', '101'), ('01', '103'), ('01', '105'), ('01', '107'), ('01', '109'), ('01', '111'), ('01', '113'), ('01', '115'), ('01', '117'), ('01', '119'), ('01', '121'), ('01', '123'), ('01', '125'), ('01', '127'), ('01', '129'), ('01', '131'), ('01', '1

Add Data to County Level GeoJSON

In [89]:
def getCoordBounds(coordList):
    if isinstance(coordList[0], list):
        min_lat = 180
        max_lat = -180
        min_long = 180
        max_long = -180
        for c in coordList:
            lat_min, lat_max, long_min, long_max = getCoordBounds(c)
            min_lat = min(min_lat, lat_min)
            max_lat = max(max_lat, lat_max)
            min_long = min(min_long, long_min)
            max_long = max(max_long, long_max)
        return (min_lat, max_lat, min_long, max_long)
    
    else:
        long = coordList[0]
        lat = coordList[1]
        return (lat, lat, long, long)

In [ ]:
for fips in tqdm(fips_list):
    county_df = df[df["county_fips"] == fips][["year", "totalvotes", "dvotes", "rvotes", "statevotes", "vote_percentage", "r_margin_perc"]]
    county_df = county_df.sort_values(by=['year'], ascending=False)
    for idx, feature in enumerate(topo["features"]):
        if int(feature["id"]) != int(fips):
            continue
        
        county = feature["properties"]
        county_data = []
        for row in county_df.itertuples():
            datapoint = {
                "year":row.year,
                "totalvotes":row.totalvotes,
                "dvotes":row.dvotes,
                "rvotes":row.rvotes,
                "vote_perc":row.vote_percentage,
                "r_marg":row.r_margin_perc
            }
            county_data.append(datapoint)
        topo["features"][idx]["properties"]["data"] = county_data
        break
    
    min_lat, max_lat, min_long, max_long = getCoordBounds(topo["features"][idx]["geometry"]["coordinates"])
    delta_lat = (max_lat - min_lat) / 2
    center_lat = (max_lat + min_lat) / 2
    delta_long = (max_long - min_long) / 2
    center_long = (max_long + min_long) / 2
    topo["features"][idx]["center"] = {
        "lat": center_lat,
        "long": center_long,
        "delta_lat": delta_lat,
        "delta_long": delta_long
    }

100%|██████████| 3172/3172 [00:03<00:00, 1021.95it/s]


In [91]:
print(topo["features"][0]["properties"])

{'GEO_ID': '0500000US01001', 'STATE': '01', 'COUNTY': '001', 'NAME': 'Autauga', 'LSAD': 'County', 'CENSUSAREA': 594.436, 'data': [{'year': 2024, 'totalvotes': 28190, 'dvotes': 7439, 'rvotes': 20484, 'vote_perc': 0.0124936180170469, 'r_marg': 0.0057814560848661}, {'year': 2020, 'totalvotes': 27770, 'dvotes': 7503, 'rvotes': 19838, 'vote_perc': 0.011952918328468, 'r_marg': 0.0053092995168042}, {'year': 2016, 'totalvotes': 24973, 'dvotes': 5936, 'rvotes': 18172, 'vote_perc': 0.0117610380117991, 'r_marg': 0.0057625459941686}, {'year': 2012, 'totalvotes': 23932, 'dvotes': 6363, 'rvotes': 17379, 'vote_perc': 0.011559381419497, 'r_marg': 0.0053208317615401}, {'year': 2008, 'totalvotes': 23641, 'dvotes': 6093, 'rvotes': 17403, 'vote_perc': 0.0112585894308033, 'r_marg': 0.005386178523006}, {'year': 2004, 'totalvotes': 20081, 'dvotes': 4758, 'rvotes': 15196, 'vote_perc': 0.010662015540919, 'r_marg': 0.0055420605655153}, {'year': 2000, 'totalvotes': 17208, 'dvotes': 4942, 'rvotes': 11993, 'vote_p

In [92]:
from geojson_rewind import rewind
topo = rewind(topo, rfc7946=False)

In [93]:
with open("../site/data/county_data.json", "w") as file:
    json.dump(topo, file, ensure_ascii=False, indent=4)

Modify State Level to have presidential Results

In [45]:
df_statelevel = pd.read_excel("Election_State_Results.xlsx", sheet_name="Summarized_Data")
df_statelevel.head()

Year  State_Fips  D_Votes  R_Votes  O_Votes   T_Votes State
0  2024           1   772412  1462616    30062   2265090    AL
1  2024           2   140026   184458    13693    338177    AK
2  2024           4  1582860  1770242    37059   3390161    AZ
3  2024           5   396905   759241    26530   1182676    AR
4  2024           6  9276179  6081697   507599  15865475    CA

In [46]:
df_statelevel["R_Perc"] = 100 * (df_statelevel["R_Votes"] / df_statelevel["T_Votes"])
df_statelevel["D_Perc"] = 100 * (df_statelevel["D_Votes"] / df_statelevel["T_Votes"])
df_statelevel["R_Margin"] = df_statelevel["R_Perc"] - df_statelevel["D_Perc"]
df_statelevel.head()

Year  State_Fips  D_Votes  R_Votes  O_Votes   T_Votes State     R_Perc  \
0  2024           1   772412  1462616    30062   2265090    AL  64.572092   
1  2024           2   140026   184458    13693    338177    AK  54.544809   
2  2024           4  1582860  1770242    37059   3390161    AZ  52.217048   
3  2024           5   396905   759241    26530   1182676    AR  64.196872   
4  2024           6  9276179  6081697   507599  15865475    CA  38.332902   

      D_Perc   R_Margin  
0  34.100720  30.471372  
1  41.406128  13.138682  
2  46.689818   5.527230  
3  33.559910  30.636962  
4  58.467704 -20.134802

In [79]:
with open("maps/us-states.json", 'r') as file:
    states_json = json.load(file)

In [81]:
for idx, feature in enumerate(states_json["features"]):
    geometry = feature["geometry"]
    min_lat, max_lat, min_long, max_long = getCoordBounds(geometry["coordinates"])
    
    delta_lat = (max_lat - min_lat) / 2
    center_lat = (max_lat + min_lat) / 2
    delta_long = (max_long - min_long) / 2
    center_long = (max_long + min_long) / 2
    states_json["features"][idx]["center"] = {
        "lat": center_lat,
        "long": center_long,
        "delta_lat": delta_lat,
        "delta_long": delta_long
    }

In [82]:
for idx, feature in enumerate(states_json["features"]):
    states_json["features"][idx]["properties"]["id"] = states_json["features"][idx]["id"]

In [83]:
for row in df_statelevel.itertuples():
    for idx, feature in enumerate(states_json["features"]):
        if int(feature["id"]) == row.State_Fips:
            datapoint = {
                "year": row.Year,
                "R%" : row.R_Perc,
                "D%" : row.D_Perc,
                "R+" : row.R_Margin,
                "RVotes": row.R_Votes,
                "DVotes": row.D_Votes,
            }

            if "data" in feature["properties"]:
                states_json["features"][idx]["properties"]["data"].append(datapoint)
            else:
                states_json["features"][idx]["properties"]["data"] = [datapoint]
            break
        
    

In [84]:
with open("../site/data/states_data.json", "w") as file:
    json.dump(states_json, file, ensure_ascii=False, indent=4)